# OGA Field Production Web Scraper

The UK OGA (Oil & Gas Authority) changed it website containing production data in 2018. Extracting the data by field and by fluid type has become fairly difficult. Consequently, I built a web scraper to extract the data in an easier fashion.

It should be highlighted that the web scraper is a template for more detailed work. That is to say, it extracts oil production by Field by Month. It could be extended to do so for gas, associated gas, and water. Furthermore, due to the structure of the website, it would require additional complexity to scrape the first 5 years on record. It would be much faster to extract these manually once (copy and paste) and then keep them in a database, where the additional data can be appended to. 

## Import Libraries

Import the most common Python libraries, along with some webscraping tools.

In [1]:
# Load common libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
%matplotlib inline
%config InlineBackend.figure_format ='retina'

In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [3]:
import functools

In [4]:
# Pandas setting to display all columns
pd.set_option('display.max_columns', None)

## Setup Web Scraper
Here we define the root URL of the OGA website that host the production data.

In [5]:
root_url = "https://itportal.ogauthority.co.uk/pprslive/production-data"

### Define Scraper Functions

In [15]:
# Function to generate required URL for each iteration.
def url_generator(y, pt):
    end_url = '?start_date=Dec+{}&production_type={}&production_unit=OIL_FIELD_UNITS&operator=ANY&regulatory_jurisdiction=ANY'.format(y,pt)
    dynamic_url = root_url + end_url
    return dynamic_url

In [16]:
# Function to scrape the URL and clean up the extracted text.
def scraper(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    prod_table = soup.find('table', class_='prod-data-table')
    prod_data = prod_table.find_all('tr')
    
    raw_data = []
    for row in range(len(prod_data)):
        val = prod_data[row].get_text().strip()
        if row == 0:
            val = val.replace('\n ',',')
            val = val.replace('\n','')
            val = val.split(',')
            raw_data.append(val)
        else:
            val = val.replace('\n',',')
            val = val.replace(',,',',')
            val = val.replace('  ','')
            val = val.split(',')
            raw_data.append(val)
    
    return raw_data

In [17]:
# Function to create a list of dataframes built from each URL interrogation.
def df_builder(year, prod_type):
    df_list = []
    for y in year:
        for pt in prod_type:
            url = url_generator(y, pt)
            data = scraper(url)
            table = data
            header = data.pop(0)
            header = [x.strip() for x in header]
            header[0] = 'Reporting Unit'
            df = pd.DataFrame(data,columns=header)
            df_list.append(df)
    
    return df_list

## Execute Web Scraper

### Build a List of Dataframes
Scrape each URL and build a dataframe. Create a list of these dataframes and inspect one to confirm the scraper is working correctly.

In [18]:
# Specify the years to pull from
year = range(1976,2018)

# Specify the fluid type
prod_type = ['OIL']

# An example of pulling more than just OIL:
# prod_type = 'OIL,DRY_GAS'.split(',')

In [19]:
dfs = df_builder(year, prod_type)

In [20]:
# Inspect the first dataframe in the list
dfs[0]

,Reporting Unit,Operator,Jan 76,Feb 76,Mar 76,Apr 76,May 76,Jun 76,Jul 76,Aug 76,Sep 76,Oct 76,Nov 76,Dec 76,Average
0,ARGYLL,BHP BILLITON PETROLEUM,0,15020,19667,26546,32306,31099,13912,33911,30282,26200,26190,16646,22648
1,AUK,REPSOL SINOPEC RESOURCES,148,7337,14202,20395,19492,31118,21730,16945,40561,43454,49672,33113,24847
2,BERYL,APACHE CORPORATION,-,-,-,-,-,651,1459,6975,14741,19152,21976,22021,12425
3,BRENT,ROYAL DUTCH SHELL,-,-,-,-,-,-,-,-,-,-,4289,24696,14492
4,FORTIES,BP EXPLORATION,73319,77941,84135,104105,125214,166420,137526,194023,214421,292885,288562,340042,174883
5,MONTROSE,REPSOL SINOPEC RESOURCES,-,-,-,-,-,88,2031,2739,3030,4314,7047,9135,4055


In order to merge all the dataframes together, we will need to create a dataframe with a master list of unique producing fields. We initially get a list of all the field names that come up and then use a set to choose a unique list.

In [37]:
# Build a list of Field names
field_names = []
for df in dfs:
    field_names.extend(df.iloc[:,0].tolist())

In [48]:
# Create a set of unique Field names sorted alphabetically
field_list = sorted(list(set(field_names)))

In [49]:
# Set up a master dataframe with unique Field names and instantiate the Operating_Co column
df_master = pd.DataFrame({'Field_Name':field_list})
df_master['Operating_Co'] = np.nan

In [52]:
# Loop through all dataframes in the dfs list and merge on the master Field column
for df_ in dfs:
    df_master = pd.merge(left=df_master, right=df_, how='left', left_on='Field_Name', right_on='Reporting Unit')
    df_master['Operating_Co'] = df_master['Operating_Co'].fillna(df_master['Operator'])
    df_master = df_master.drop(['Operator','Reporting Unit', 'Average'], axis=1)

In [53]:
# Data cleanup of null values
df_master.iloc[:,2:] = df_master.iloc[:,2:].replace('-',np.nan)

In [55]:
# Final product
df_master.head()

,Field_Name,Operating_Co,Jan 76,Feb 76,Mar 76,Apr 76,May 76,Jun 76,Jul 76,Aug 76,Sep 76,Oct 76,Nov 76,Dec 76,Jan 77,Feb 77,Mar 77,Apr 77,May 77,Jun 77,Jul 77,Aug 77,Sep 77,Oct 77,Nov 77,Dec 77,Jan 78,Feb 78,Mar 78,Apr 78,May 78,Jun 78,Jul 78,Aug 78,Sep 78,Oct 78,Nov 78,Dec 78,Jan 79,Feb 79,Mar 79,Apr 79,May 79,Jun 79,Jul 79,Aug 79,Sep 79,Oct 79,Nov 79,Dec 79,Jan 80,Feb 80,Mar 80,Apr 80,May 80,Jun 80,Jul 80,Aug 80,Sep 80,Oct 80,Nov 80,Dec 80,Jan 81,Feb 81,Mar 81,Apr 81,May 81,Jun 81,Jul 81,Aug 81,Sep 81,Oct 81,Nov 81,Dec 81,Jan 82,Feb 82,Mar 82,Apr 82,May 82,Jun 82,Jul 82,Aug 82,Sep 82,Oct 82,Nov 82,Dec 82,Jan 83,Feb 83,Mar 83,Apr 83,May 83,Jun 83,Jul 83,Aug 83,Sep 83,Oct 83,Nov 83,Dec 83,Jan 84,Feb 84,Mar 84,Apr 84,May 84,Jun 84,Jul 84,Aug 84,Sep 84,Oct 84,Nov 84,Dec 84,Jan 85,Feb 85,Mar 85,Apr 85,May 85,Jun 85,Jul 85,Aug 85,Sep 85,Oct 85,Nov 85,Dec 85,Jan 86,Feb 86,Mar 86,Apr 86,May 86,Jun 86,Jul 86,Aug 86,Sep 86,Oct 86,Nov 86,Dec 86,Jan 87,Feb 87,Mar 87,Apr 87,May 87,Jun 87,Jul 87,Aug 87,Sep 87,Oct 87,Nov 87,Dec 87,Jan 88,Feb 88,Mar 88,Apr 88,May 88,Jun 88,Jul 88,Aug 88,Sep 88,Oct 88,Nov 88,Dec 88,Jan 89,Feb 89,Mar 89,Apr 89,May 89,Jun 89,Jul 89,Aug 89,Sep 89,Oct 89,Nov 89,Dec 89,Jan 90,Feb 90,Mar 90,Apr 90,May 90,Jun 90,Jul 90,Aug 90,Sep 90,Oct 90,Nov 90,Dec 90,Jan 91,Feb 91,Mar 91,Apr 91,May 91,Jun 91,Jul 91,Aug 91,Sep 91,Oct 91,Nov 91,Dec 91,Jan 92,Feb 92,Mar 92,Apr 92,May 92,Jun 92,Jul 92,Aug 92,Sep 92,Oct 92,Nov 92,Dec 92,Jan 93,Feb 93,Mar 93,Apr 93,May 93,Jun 93,Jul 93,Aug 93,Sep 93,Oct 93,Nov 93,Dec 93,Jan 94,Feb 94,Mar 94,Apr 94,May 94,Jun 94,Jul 94,Aug 94,Sep 94,Oct 94,Nov 94,Dec 94,Jan 95,Feb 95,Mar 95,Apr 95,May 95,Jun 95,Jul 95,Aug 95,Sep 95,Oct 95,Nov 95,Dec 95,Jan 96,Feb 96,Mar 96,Apr 96,May 96,Jun 96,Jul 96,Aug 96,Sep 96,Oct 96,Nov 96,Dec 96,Jan 97,Feb 97,Mar 97,Apr 97,May 97,Jun 97,Jul 97,Aug 97,Sep 97,Oct 97,Nov 97,Dec 97,Jan 98,Feb 98,Mar 98,Apr 98,May 98,Jun 98,Jul 98,Aug 98,Sep 98,Oct 98,Nov 98,Dec 98,Jan 99,Feb 99,Mar 99,Apr 99,May 99,Jun 99,Jul 99,Aug 99,Sep 99,Oct 99,Nov 99,Dec 99,Jan 00,Feb 00,Mar 00,Apr 00,May 00,Jun 00,Jul 00,Aug 00,Sep 00,Oct 00,Nov 00,Dec 00,Jan 01,Feb 01,Mar 01,Apr 01,May 01,Jun 01,Jul 01,Aug 01,Sep 01,Oct 01,Nov 01,Dec 01,Jan 02,Feb 02,Mar 02,Apr 02,May 02,Jun 02,Jul 02,Aug 02,Sep 02,Oct 02,Nov 02,Dec 02,Jan 03,Feb 03,Mar 03,Apr 03,May 03,Jun 03,Jul 03,Aug 03,Sep 03,Oct 03,Nov 03,Dec 03,Jan 04,Feb 04,Mar 04,Apr 04,May 04,Jun 04,Jul 04,Aug 04,Sep 04,Oct 04,Nov 04,Dec 04,Jan 05,Feb 05,Mar 05,Apr 05,May 05,Jun 05,Jul 05,Aug 05,Sep 05,Oct 05,Nov 05,Dec 05,Jan 06,Feb 06,Mar 06,Apr 06,May 06,Jun 06,Jul 06,Aug 06,Sep 06,Oct 06,Nov 06,Dec 06,Jan 07,Feb 07,Mar 07,Apr 07,May 07,Jun 07,Jul 07,Aug 07,Sep 07,Oct 07,Nov 07,Dec 07,Jan 08,Feb 08,Mar 08,Apr 08,May 08,Jun 08,Jul 08,Aug 08,Sep 08,Oct 08,Nov 08,Dec 08,Jan 09,Feb 09,Mar 09,Apr 09,May 09,Jun 09,Jul 09,Aug 09,Sep 09,Oct 09,Nov 09,Dec 09,Jan 10,Feb 10,Mar 10,Apr 10,May 10,Jun 10,Jul 10,Aug 10,Sep 10,Oct 10,Nov 10,Dec 10,Jan 11,Feb 11,Mar 11,Apr 11,May 11,Jun 11,Jul 11,Aug 11,Sep 11,Oct 11,Nov 11,Dec 11,Jan 12,Feb 12,Mar 12,Apr 12,May 12,Jun 12,Jul 12,Aug 12,Sep 12,Oct 12,Nov 12,Dec 12,Jan 13,Feb 13,Mar 13,Apr 13,May 13,Jun 13,Jul 13,Aug 13,Sep 13,Oct 13,Nov 13,Dec 13,Jan 14,Feb 14,Mar 14,Apr 14,May 14,Jun 14,Jul 14,Aug 14,Sep 14,Oct 14,Nov 14,Dec 14,Jan 15,Feb 15,Mar 15,Apr 15,May 15,Jun 15,Jul 15,Aug 15,Sep 15,Oct 15,Nov 15,Dec 15,Jan 16,Feb 16,Mar 16,Apr 16,May 16,Jun 16,Jul 16,Aug 16,Sep 16,Oct 16,Nov 16,Dec 16,Jan 17,Feb 17,Mar 17,Apr 17,May 17,Jun 17,Jul 17,Aug 17,Sep 17,Oct 17,Nov 17,Dec 17
0,AFFLECK,MAERSK GROUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### Potential Modifications to the Dataframe
The dataframe could be transposed to store it with the index as month/year and the features as the Producing Field. This would make plotting easier as well as handling the data as a time series. This would probably work better with DCA tools and probably with OFM. Unfortunately, some metadata is lost, such as Operating Company.

In [71]:
df_prod = df_master.drop(['Operating_Co'], axis=1)

In [72]:
df_prod = df_prod.transpose()

In [73]:
# Format column headers correctly
df_prod.columns = df_prod.iloc[0]
df_prod = df_prod.drop('Field_Name')

In [74]:
df_prod.head()

Field_Name,AFFLECK,ALBA,ALDER,ALMA,ALWYN NORTH,ANDREW,ANGUS,ARBROATH,ARDMORE,ARGYLL,ARKWRIGHT,ARUNDEL,ATHENA,ATLANTIC,AUK,AUK NORTH,AVIAT,AVINGTON,BACCHUS,BALLOCH,BALMORAL,BANFF,BARDOLINO,BARRA,BEATRICE,BEAULY,BECKINGHAM,BECKINGHAM WEST,BEINN,BERYL,BIRCH,BITTERN,BLACKBIRD,BLADON,BLAIR,BLAKE,BLANE,BLENHEIM,BLETCHINGLEY,BOA,BOTHAMSALL,BRAE CENTRAL,BRAE NORTH,BRAE SOUTH,BRAEMAR,BRECHIN,BRENDA,BRENT,BRIMMOND,BRITANNIA,BROCKHAM,BRODGAR,BROOM,BRUCE,BUCHAN,BUCKLAND,BURGHLEY,BURGMAN,BUZZARD,CALEDONIA,CALLANISH,CALLATER,CAPTAIN,CARNOUSTIE,CATCHER,CAUSEWAY,CAYLEY,CHANTER,CHESTNUT,CLADHAN,CLAIR-PHASE 1,CLAPHAM,CLAYMORE,CLEETON,CLYDE,COLD HANWORTH,COLUMBA B,COLUMBA BD,COLUMBA D,COLUMBA E,CONRIE,CONWY,COOK,CORMORANT EAST,CORMORANT NORTH,CORRINGHAM,CRATHES,CRAWFORD,CROSBY WARREN,CURLEW,CURLEW C,CYRUS,DAUNTLESS,DEVENICK,DEVERON,DON,DON SOUTH WEST,DONAN,DONAN MAERSK,DOUGLAS,DOUGLAS WEST,DRAKE,DUART,DUKES WOOD,DUNBAR,DUNCAN,DUNLIN,DUNLIN SOUTH WEST,DURWARD,EAST BRAE,EAST GLENTWORTH,EDRADOUR,EGMANTON,EGRET,EIDER,ELGIN,ELLON,EMERALD,ENOCH,ENOCHDHU,ERSKINE,ESMOND,ETTRICK,EVEREST,FALCON,FARLEYS WOOD,FARRAGON,FERGUS,FIFE,FIONN,FISKERTON AIRFIELD,FLEMING,FLORA,FLYNDRE,FOINAVEN,FORBES,FORTIES,FORVIE NORTH,FRANKLIN,FULMAR,GADWALL,GAINSBOROUGH,GALIA,GALLEY,GANNET A,GANNET B,GANNET C,GANNET D,GANNET E,GANNET F,GANNET G,GLAMIS,GLENELG,GLENLIVET,GLENTWORTH,GODWIN,GOLDEN EAGLE,GOLDENEYE,GOODWORTH,GOOSANDER,GORDON,GRANT,GROUSE,GRYPHON,GUILLEMOT A,GUILLEMOT NORTH WEST,GUILLEMOT WEST,HALLEY,HAMISH,HANNAY,HARDING,HARRIS,HAWKINS,HEATHER,HERON,HERRIARD,HIGHLANDER,HORNDEAN,HOWE,HUDSON,HUMBLY GROVE,HUNTINGTON,HUTTON,INNES,IONA,ISLAY,IVANHOE,JACKY,JADE,JAMES,JANICE,JASMINE,JOANNE,JUDY,JURA,KEDDINGTON,KEITH,KESTREL,KIMMERIDGE,KINGFISHER,KINNOULL,KIRKLINGTON,KITTIWAKE,KRAKEN,KRAKEN NORTH,KYLE,LAGGAN,LARCH,LEADON,LENNOX,LEVEN,LIDSEY,LINNHE,LOCHRANZA,LOIRSTON,LOMOND,LONG CLAWSON,LOYAL,LYBSTER,LYELL,MACCULLOCH,MACHAR,MACLURE,MADOES,MAGNUS,MAGNUS SOUTH,MALLARD,MARIA,MARNOCK,MAULE,MAUREEN,MEDWIN,MERGANSER,MERLIN,MILLER,MIRREN,MOIRA,MONAN,MONTROSE,MUNGO,MURCHISON,NELSON,NESS,NETHAN,NETTLEHAM,NEVIS,NEWTON-ON-TRENT,NICOL,NINIAN,NORTHWEST HUTTON,ORION,OSPREY,OTTER,PALMERS WOOD,PELICAN,PENGUIN EAST,PENGUIN WEST,PEREGRINE,PETRONELLA,PICT,PIERCE,PIPER,PLAYFAIR,REMPSTONE,RENEE,RHUM,ROB ROY,ROCHELLE,ROSS,RUBIE,SALTIRE,SAXON,SCAMPTON,SCAMPTON NORTH,SCAPA,SCHIEHALLION,SCOLTY,SCOTER,SCOTT,SEDGWICK,SEYMOUR,SHAW,SHEARWATER,SHELLEY,SINGLETON,SKENE,SKUA,SOLAN,SOLITAIRE,SOUTH CORMORANT,SOUTH LEVERTON,STAFFA,STAINTON,STARLING,STATFJORD,STELLA,STIRLING,STOCKBRIDGE,STORRINGTON,STRATHSPEY,SYCAMORE,TARTAN,TEAL,TEAL SOUTH,TELFORD,TERN,THELMA,THISTLE,TIFFANY,TONI,TONTO,TORMORE,TULLICH,TWEEDSMUIR,TWEEDSMUIR SOUTH,VARADERO,WADDOCK CROSS,WAREHAM,WELTON,WEST BRAE,WEST DON,WEST FIRSBY,WHISBY,WOOD,WYTCH FARM,YTHAN
Jan 76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN